<a href="https://colab.research.google.com/github/sarahgruetz/AluraChallenge_DS_2/blob/main/Alura_Challenge_semanas_3_e_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 38.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3d5d947076dce73f0ddbef2757c3ebd967b7253b7c280f7ab323b100ac664b3b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


### Iniciando a SparkSession

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Alura Challenge") \
    .getOrCreate()

In [3]:
import zipfile

zipfile.ZipFile('/content/drive/MyDrive/curso-spark/ChallengeAlura/semanas-3-e-4.zip','r')\
        .extractall('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados')

In [4]:
dataset = spark.read.parquet('/content/drive/MyDrive/curso-spark/ChallengeAlura/dados/dataset_ml_parquet')

In [10]:
print(f'O dataset possui {dataset.count()} registros e {len(dataset.columns)} colunas.')

O dataset possui 66551 registros e 25 colunas.


In [5]:
dataset.show(5)

+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|                  id|andar|area_util|banheiros|quartos|suites|vaga|      bairro|condominio| iptu|   valor|Zona Central|Zona Norte|Zona Oeste|Zona Sul|Academia|Animais permitidos|Churrasqueira|Condomínio fechado|Elevador|Piscina|Playground|Portaria 24h|Portão eletrônico|Salão de festas|
+--------------------+-----+---------+---------+-------+------+----+------------+----------+-----+--------+------------+----------+----------+--------+--------+------------------+-------------+------------------+--------+-------+----------+------------+-----------------+---------------+
|00002dd9-cc74-480...|    2|       35|        1|      1|   0.0| 0.0|Santo Cristo|     100.0|100.0|245000.0|           1|         0|     

In [6]:
dataset.printSchema()

root
 |-- id: string (nullable = true)
 |-- andar: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: double (nullable = true)
 |-- vaga: double (nullable = true)
 |-- bairro: string (nullable = true)
 |-- condominio: double (nullable = true)
 |-- iptu: double (nullable = true)
 |-- valor: double (nullable = true)
 |-- Zona Central: integer (nullable = true)
 |-- Zona Norte: integer (nullable = true)
 |-- Zona Oeste: integer (nullable = true)
 |-- Zona Sul: integer (nullable = true)
 |-- Academia: integer (nullable = true)
 |-- Animais permitidos: integer (nullable = true)
 |-- Churrasqueira: integer (nullable = true)
 |-- Condomínio fechado: integer (nullable = true)
 |-- Elevador: integer (nullable = true)
 |-- Piscina: integer (nullable = true)
 |-- Playground: integer (nullable = true)
 |-- Portaria 24h: integer (nullable = true)
 |-- Portão eletrônico: integer (nullable 

In [12]:
from pyspark.sql import functions as f

In [13]:
dataset\
  .select([f.count(f.when(f.isnan(c) | f.isnull(c), True)).alias(c) for c in dataset.columns])\
  .show(vertical=True)

-RECORD 0-----------------
 id                 | 0   
 andar              | 0   
 area_util          | 0   
 banheiros          | 0   
 quartos            | 0   
 suites             | 0   
 vaga               | 0   
 bairro             | 0   
 condominio         | 0   
 iptu               | 0   
 valor              | 0   
 Zona Central       | 0   
 Zona Norte         | 0   
 Zona Oeste         | 0   
 Zona Sul           | 0   
 Academia           | 0   
 Animais permitidos | 0   
 Churrasqueira      | 0   
 Condomínio fechado | 0   
 Elevador           | 0   
 Piscina            | 0   
 Playground         | 0   
 Portaria 24h       | 0   
 Portão eletrônico  | 0   
 Salão de festas    | 0   



### Preparando os dados

In [7]:
from pyspark.ml.feature import VectorAssembler

In [14]:
dataset = dataset.withColumnRenamed('valor','label')

In [15]:
dataset.columns

['id',
 'andar',
 'area_util',
 'banheiros',
 'quartos',
 'suites',
 'vaga',
 'bairro',
 'condominio',
 'iptu',
 'label',
 'Zona Central',
 'Zona Norte',
 'Zona Oeste',
 'Zona Sul',
 'Academia',
 'Animais permitidos',
 'Churrasqueira',
 'Condomínio fechado',
 'Elevador',
 'Piscina',
 'Playground',
 'Portaria 24h',
 'Portão eletrônico',
 'Salão de festas']

In [16]:
X = [
    'andar',
    'area_util',
    'banheiros',
    'quartos',
    'suites',
    'vaga',
    'condominio',
    'iptu',
    'Zona Central',
    'Zona Norte',
    'Zona Oeste',
    'Zona Sul',
    'Academia',
    'Animais permitidos',
    'Churrasqueira',
    'Condomínio fechado',
    'Elevador',
    'Piscina',
    'Playground',
    'Portaria 24h',
    'Portão eletrônico',
    'Salão de festas'
]

In [17]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [18]:
dataset_modelo = assembler.transform(dataset).select('features','label')

In [20]:
dataset_modelo.show(5, truncate=False)

+------------------------------------------------------------------------------------------------+--------+
|features                                                                                        |label   |
+------------------------------------------------------------------------------------------------+--------+
|[2.0,35.0,1.0,1.0,0.0,0.0,100.0,100.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0]  |245000.0|
|(22,[0,1,2,3,5,6,7,9,14,16,18,19,21],[1.0,84.0,2.0,2.0,1.0,770.0,105.0,1.0,1.0,1.0,1.0,1.0,1.0])|474980.0|
|(22,[1,2,3,6,7,11,13,16],[85.0,2.0,2.0,460.0,661.0,1.0,1.0,1.0])                                |290000.0|
|(22,[1,2,3,5,6,7,10,17,18],[58.0,1.0,2.0,1.0,550.0,550.0,1.0,1.0,1.0])                          |249000.0|
|(22,[1,2,3,4,5,6,9],[64.0,2.0,2.0,1.0,1.0,850.0,1.0])                                           |530000.0|
+------------------------------------------------------------------------------------------------+--------+
only showing top 5 rows

